In [2]:
import numpy as np
import pandas as pd
import os
import json
from dateutil import parser
from zipfile import ZipFile

In [3]:

macro = pd.read_csv('macro.csv', index_col=0)  
exp = pd.read_csv('exp.csv', index_col=0)  
run = pd.read_csv('run.csv', index_col=0)  

In [6]:
%pwd

'C:\\Users\\tomglove_b\\Documents\\GitHub\\DeepCA---Hybrid-Deep-Learning-Cellular-Automata-Reservoir\\ReCA parameter space\\Data'

In [7]:
%%time

def paramsToDict(params):
    dictP = {}
    for pa in params:
        arrayP = pa.split("=")
        if len(arrayP) != 2:
            arrayP = ["started at", "".join(arrayP[0].split(":")[1:5]).replace("\n", "")]
#         print(arrayP)
        dictP[arrayP[0].strip()] = arrayP[1]
    return dictP

from zipfile import ZipFile
directory = "import"
with ZipFile(os.path.join(directory, "D Period and RWidth R1, I1 Rule 54.zip"), "r") as zip:
    for filename in zip.namelist():
        with zip.open(filename) as file:
            lines = file.readlines()
            params = (lines[0].decode("utf-8")).split(",")
            dictP = paramsToDict(params)
#                 print(dictP)
            ex = {
                "Nb": int(dictP.get("bits", 5)),
                "R" : int(dictP.get("r", -1)),
                "I" : int(dictP.get("itr", -1)),
                "Ld": int(dictP.get("r width")) if dictP.get("r width") != None else int(dictP.get("r total width", -1)) / int(dictP.get("r", -1)) ,
                "Dp": int(dictP.get("distractor period", -1)),
                "CA": int(dictP.get("CA rule", -1)),
                "Nruns": int(dictP.get("number of runs", -1)),
                "StartTime": parser.parse(dictP.get("started at", -1)),
            }
            exIndex = exp.index.size
            macroExp = macro[(macro["Nb"]==ex["Nb"])
                     & (macro["R"]==ex["R"])
                     & (macro["I"]==ex["I"])
                     & (macro["Ld"]==ex["Ld"])
                     & (macro["Dp"]==ex["Dp"])
                        ]
            macroIndex = None
            if(macroExp.empty):
                macroIndex = macro.index.size
                macro = macro.append({"Benchmark":"X-Bit", "Nb": ex["Nb"],"R": ex["R"],"I": ex["I"],"Ld": ex["Ld"],"Dp": ex["Dp"]}, ignore_index=True)
            else:
                macroIndex = macroExp.index.max()
            exp =exp.append({"macroExperiment": macroIndex, "CARule":ex["CA"], "StartTime": ex["StartTime"]}, ignore_index=True)

            runs = lines[2:ex["Nruns"]+2]
            for inputRun in runs:
                inputRun = inputRun.decode("utf-8")
                Score = float(inputRun.split("\t")[0])
                map = json.loads((inputRun.split("\t")[1]).replace("\n", ""))
                run = run.append({"Experiment": exIndex, "Avg":Score, "RandomMapping":map}, ignore_index=True)

    

Wall time: 2min 12s


In [8]:
run

,Experiment,Avg,RandomMapping
0,0,0.954545,"[30, 8, 1, 35, 61, 49, 40, 45, 93, 101, 94, 11..."
1,0,0.954545,"[24, 13, 38, 5, 61, 49, 40, 69, 94, 100, 111, ..."
2,0,0.954545,"[6, 20, 36, 11, 46, 75, 43, 52, 91, 117, 103, ..."
3,0,0.954545,"[13, 32, 6, 39, 64, 46, 45, 71, 109, 115, 80, ..."
4,0,0.954545,"[8, 21, 29, 10, 52, 45, 51, 54, 115, 111, 85, ..."
...,...,...,...
405495,4054,0.976744,"[7, 27, 5, 13]"
405496,4054,0.980669,"[23, 15, 16, 38]"
405497,4054,0.987064,"[29, 33, 9, 5]"
405498,4054,0.988663,"[15, 36, 14, 40]"


In [9]:
exp[3934-121: 3934]

,macroExperiment,CARule,StartTime
3813,4,150,2021-06-29 16:18:22.882765
3814,5,150,2021-06-29 16:50:22.525532
3815,6,150,2021-06-29 17:22:29.829041
3816,7,150,2021-06-29 17:54:45.206736
3817,8,150,2021-06-29 18:27:11.019688
...,...,...,...
3929,120,150,2021-07-02 08:02:04.213963
3930,121,150,2021-07-02 08:35:33.221767
3931,122,150,2021-07-02 09:09:52.717239
3932,123,150,2021-07-02 09:44:18.249977


In [10]:
macro[120:125]

,Benchmark,R,I,Dp,Ld,Nb
120,X-Bit,1,1,201,45.0,5
121,X-Bit,1,1,202,45.0,5
122,X-Bit,1,1,203,45.0,5
123,X-Bit,1,1,204,45.0,5
124,X-Bit,1,1,205,45.0,5


In [11]:
macro.to_csv("macro.csv")
exp.to_csv("exp.csv")
run.to_csv("run.csv")